# Building Machine Learning Classifiers: Explore Gradient Boosting model with grid-search

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

### Read in & clean text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])

X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
X_features.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Explore GradientBoostingClassifier Attributes & Hyperparameters

In [2]:
from sklearn.ensemble import GradientBoostingClassifier

In [4]:
print(dir(GradientBoostingClassifier))
print(GradientBoostingClassifier().get_params()) # .get_params() method to show hyperparameters

# Note: In the hyperparameters, there is no "n_jobs" because Gradient Boosting can't be parallelized

['_SUPPORTED_LOSS', '__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_check_initialized', '_check_n_features', '_check_params', '_clear_state', '_compute_partial_dependence_recursion', '_estimator_type', '_fit_stage', '_fit_stages', '_get_param_names', '_get_tags', '_init_state', '_is_initialized', '_make_estimator', '_more_tags', '_raw_predict', '_raw_predict_init', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_required_parameters', '_resize_state', '_staged_raw_predict', '_validate_data', '_validate_estimator', '_validate_y', '_warn_mae_for_criterion', 'apply

### Build our own Grid-search

In [5]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

In [7]:
def train_GB(est, max_depth, lr):
    gb = GradientBoostingClassifier(n_estimators=est, max_depth=max_depth, learning_rate=lr)
    gb_model = gb.fit(X_train, y_train)
    y_pred = gb_model.predict(X_test)
    precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')
    print('Est: {} / Depth: {} / LR: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
        est, max_depth, lr, round(precision, 3), round(recall, 3), 
        round((y_pred==y_test).sum() / len(y_pred), 3)))

In [8]:
for n_est in [50, 100, 150]:
    for max_depth in [3, 7, 11, 15]:
        for lr in [0.01, 0.1, 1]:
            train_GB(n_est, max_depth, lr)

c:\users\adrian\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Est: 50 / Depth: 3 / LR: 0.01 ---- Precision: 0.0 / Recall: 0.0 / Accuracy: 0.869
Est: 50 / Depth: 3 / LR: 0.1 ---- Precision: 0.957 / Recall: 0.76 / Accuracy: 0.964
Est: 50 / Depth: 3 / LR: 1 ---- Precision: 0.898 / Recall: 0.781 / Accuracy: 0.96
Est: 50 / Depth: 7 / LR: 0.01 ---- Precision: 1.0 / Recall: 0.021 / Accuracy: 0.872
Est: 50 / Depth: 7 / LR: 0.1 ---- Precision: 0.912 / Recall: 0.849 / Accuracy: 0.969
Est: 50 / Depth: 7 / LR: 1 ---- Precision: 0.868 / Recall: 0.808 / Accuracy: 0.959
Est: 50 / Depth: 11 / LR: 0.01 ---- Precision: 1.0 / Recall: 0.014 / Accuracy: 0.871
Est: 50 / Depth: 11 / LR: 0.1 ---- Precision: 0.91 / Recall: 0.829 / Accuracy: 0.967
Est: 50 / Depth: 11 / LR: 1 ---- Precision: 0.91 / Recall: 0.836 / Accuracy: 0.968


c:\users\adrian\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Est: 50 / Depth: 15 / LR: 0.01 ---- Precision: 0.0 / Recall: 0.0 / Accuracy: 0.869
Est: 50 / Depth: 15 / LR: 0.1 ---- Precision: 0.919 / Recall: 0.849 / Accuracy: 0.97
Est: 50 / Depth: 15 / LR: 1 ---- Precision: 0.911 / Recall: 0.842 / Accuracy: 0.969
Est: 100 / Depth: 3 / LR: 0.01 ---- Precision: 0.95 / Recall: 0.521 / Accuracy: 0.934
Est: 100 / Depth: 3 / LR: 0.1 ---- Precision: 0.967 / Recall: 0.795 / Accuracy: 0.969
Est: 100 / Depth: 3 / LR: 1 ---- Precision: 0.929 / Recall: 0.801 / Accuracy: 0.966
Est: 100 / Depth: 7 / LR: 0.01 ---- Precision: 0.943 / Recall: 0.678 / Accuracy: 0.952
Est: 100 / Depth: 7 / LR: 0.1 ---- Precision: 0.919 / Recall: 0.856 / Accuracy: 0.971
Est: 100 / Depth: 7 / LR: 1 ---- Precision: 0.878 / Recall: 0.836 / Accuracy: 0.963
Est: 100 / Depth: 11 / LR: 0.01 ---- Precision: 0.909 / Recall: 0.753 / Accuracy: 0.958
Est: 100 / Depth: 11 / LR: 0.1 ---- Precision: 0.906 / Recall: 0.856 / Accuracy: 0.969
Est: 100 / Depth: 11 / LR: 1 ---- Precision: 0.931 / Recall:

Note: The grid search using Gradient Boosting actually took around 1 hr and 30 mins.